In [ ]:
using Plots, LinearAlgebra, MatrixDepot, IterativeSolvers

# MTH8107 Mathématiques de l'apprentissage Profond : Devoir 2
## Bertrand Toutée 1805477 et Victor Gaudreau-Blouin 

### Question 1

#### a)

On commence par construire les matrices du système linéaire voulu :

In [ ]:
lambda = @. 10 + (1:100)
A = triu(rand(100,100),1) + diagm(0=>lambda)
b = rand(100)

In [ ]:
# On vérifie visuellement que que A est bien triangulaire supérieur 100x100
A

On génère la base de Krylov jusqu'à  $m=30$

In [ ]:
m = 30
ki = zeros(100)
Km = zeros(100,30)
for i in 1:m
    ki = A^(i-1) * b  # comme la première colonne doit être b seul alors on commence l'exposant à 0
    Km[:,i] = ki/norm(ki)  # on normalise chaque colonne
end

In [ ]:
# On vérifie que Km est bien une matrice nxm
Km

#### b)

L'opérateur \\(A,b) va nous donnez le meilleur $\vec{x}$ selon les moindre carré. Ici notre matrice A est plutôt celle obtenue en multipliant A par la base de Krylov de la taille voulue.
On calcule le résidu pour chaque taille de base qu'on utilise et que l'on garde dans un vecteur $res$.

In [ ]:
res = zeros(30)
z = zeros(100)
for i in 1:m
    z = \(A*Km[:, 1:i],b)
    res[i] = norm(b - A*Km[:, 1:i]*z) 
end

In [ ]:
# On vérifie qu'on a bien un vecteur de taille m
res

In [ ]:
# On affiche le résidu dans un graphique avec une échelle log
plot(1:m, res, title="Résidu selon la dimension de la base de Krylov", minorgrid=true)
xlabel!("Dimension de la base de Krylov")
plot!(yaxis = ("Résidu b-Ax", :log10))

Comme on s'y attendrait, on voit que plus une utilise une grande base de Krylov, moins l'erreur de notre approximation est élevée. Cette dimminution du résidu est d'ailleurs assez rapide jusqu'à m=17, étant divisé par 100 entre m=3 et m=13.
L'amélioration se fait moins marqué par la suite et semblerait peut-être même stagner. On voit donc que faire le calcul complet de dimension 100 n'est pas du tout nécessaire pour une erreur négligeable avec cette méthode.

#### c)

On calcule à nouveau le résidu selon la dimension de la base, mais cette fois ci en approximant A par la méthode de Arnoldi plutôt que la base de Krylov.

In [ ]:
# La fonction fournie dans l'énoncé
function arnoldi(A,u,m)

# Application de la méthode d'Arnoldi sur la matrice A de dimensions m x n, qui
# forme un sous-espace de dimension m, en partant du vecteur initial u de
# dimension n.
# Retourne la base orthonormale de dimension m+1 et la matrice de Hessenberg
# supérieure H de dimensions (m+1) x m.

n = length(u)
Q = zeros(n,m+1)
H = zeros(m+1,m)
Q[:,1] = u/norm(u)
for j = 1:m
  v = A*Q[:,j]
  for i = 1:j
    H[i,j] = dot(Q[:,i],v)
    v -= H[i,j]*Q[:,i]
  end
  H[j+1,j] = norm(v)
  Q[:,j+1] = v/H[j+1,j]
end

return Q,H
end

In [ ]:
# On obtient les matrices avec m=60
m = 60
u = rand(100)
Q,H = arnoldi(A,u,m)
# On obtient notre matrice à utiliser avec notre base 
A_arnoldi = Q*H*Q[:, 1:m]'

In [ ]:
# On calcule le résidu selon les colonnes de la base qu'on utilise
res_arnold = zeros(m)
z = zeros(100)
for i in 1:m
    z = \(A_arnoldi[:, 1:i],b)
    res_arnold[i] = norm(b - A_arnoldi[:, 1:i]*z) 
end

In [ ]:
# On affiche les résultats sur un graphique
plot(1:60, res_arnold, minorgrid=true, title="Résidu avec la méthode d'Arnoldi")
xlabel!("Dimension de la base")
plot!(yaxis = ("Résidu b-Ax", :log10))

Comme on s'y attend, le résidu dimminue avec l'augmentation de la taille de notre base. Cependant, le résidu est plus élevé et dimminue plus lentement qu'avec la méthode de la base de Krylov. On retrouve une erreur avec la base de dimension 60 similaire à celle de dimension faible avec la première méthode. 

### Question 2

### Question 3